In [1]:
## This Notebook takes raw IMDb title data, filters by movie, joins it with ratings data, filters
## for only movies with 100 or more total ratings, and separates by genre.

import pandas as pd
pd.set_option('display.max_columns', None)

ratingsDataset = '/Users/davidkatilius/Documents/Raw_Datasets/IMDB/title.ratings.tsv'
titleDataset = '/Users/davidkatilius/Documents/Raw_Datasets/IMDB/title.basics.tsv'

dfTitles = pd.read_csv(titleDataset, sep='\t')
dfRatings = pd.read_csv(ratingsDataset, sep='\t')

dfMovies = dfTitles[(dfTitles['titleType'] == 'movie') & (dfTitles['startYear'] != '\\N')]
dfRatedMovies = pd.merge(dfMovies, dfRatings, on = "tconst", how = "inner")

dfRatedMovies.pop('titleType')
dfRatedMovies.pop('endYear')

dfRatedMovies[['genre1', 'genre2', 'genre3']] = dfRatedMovies['genres'].str.split(pat=',', expand=True)

dfRated2 = pd.melt(dfRatedMovies, id_vars=['tconst','primaryTitle','originalTitle','isAdult','startYear','runtimeMinutes','genres','averageRating','numVotes'])
dfRated3 = dfRated2.sort_values(by=['tconst'])
dfRated3.pop('variable')

/var/folders/vb/t35ftt3d30b0gf_9v37z7st80000gn/T/ipykernel_48695/1518117966.py:10: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  dfTitles = pd.read_csv(titleDataset, sep='\t')


0         genre1
279919    genre2
559838    genre3
1         genre1
279920    genre2
           ...  
559836    genre2
279917    genre1
279918    genre1
559837    genre2
839756    genre3
Name: variable, Length: 839757, dtype: object

In [17]:
pd.options.display.max_rows = None

In [2]:
dfRated3.head()

,tconst,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes,value
0,tt0000009,Miss Jerry,Miss Jerry,0,1894,45,Romance,5.3,198,Romance
279919,tt0000009,Miss Jerry,Miss Jerry,0,1894,45,Romance,5.3,198,None
559838,tt0000009,Miss Jerry,Miss Jerry,0,1894,45,Romance,5.3,198,None
1,tt0000502,Bohemios,Bohemios,0,1905,100,\N,4.5,14,\N
279920,tt0000502,Bohemios,Bohemios,0,1905,100,\N,4.5,14,None


In [3]:
dfRatedMovies.head()

,tconst,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes,genre1,genre2,genre3
0,tt0000009,Miss Jerry,Miss Jerry,0,1894,45,Romance,5.3,198,Romance,None,None
1,tt0000502,Bohemios,Bohemios,0,1905,100,\N,4.5,14,\N,None,None
2,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,70,"Action,Adventure,Biography",6.0,778,Action,Adventure,Biography
3,tt0000591,The Prodigal Son,L'enfant prodigue,0,1907,90,Drama,4.7,19,Drama,None,None
4,tt0000615,Robbery Under Arms,Robbery Under Arms,0,1907,\N,Drama,4.5,23,Drama,None,None


In [23]:
df2 = dfRatedMovies.groupby(['startYear'], as_index = False)['numVotes'].count()

In [30]:
df2.head()

,startYear,numVotes
0,1894,1
1,1905,1
2,1906,1
3,1907,2
4,1908,3


In [25]:
df3 = dfRatedMovies.groupby(['startYear'], as_index = False)['numVotes'].sum()

In [31]:
df3.head()

,startYear,numVotes
0,1894,198
1,1905,14
2,1906,778
3,1907,42
4,1908,110


In [32]:
df4 = dfRatedMovies.groupby(['startYear'], as_index = False)['numVotes'].max()

In [33]:
df4.head()

,startYear,numVotes
0,1894,198
1,1905,14
2,1906,778
3,1907,23
4,1908,66


In [34]:
df5 = pd.merge(df2, df3, on = "startYear", how = "inner")
df5.head()

,startYear,numVotes_x,numVotes_y
0,1894,1,198
1,1905,1,14
2,1906,1,778
3,1907,2,42
4,1908,3,110


In [35]:
df6 = pd.merge(df5, df4, on = "startYear", how = "inner")
df6.head()

,startYear,numVotes_x,numVotes_y,numVotes
0,1894,1,198,198
1,1905,1,14,14
2,1906,1,778,778
3,1907,2,42,23
4,1908,3,110,66


In [36]:
df6.to_csv(r'ratings-by-year.csv', index=False)